In [1]:
LL = [3, 3]
N = LL[1]*LL[2]
vv_lattice = [ [1., 0], [0, 1.] ]
b1=[0, 0]
vv_basis = [b1]
t=-1
p=1
q=LL[2]
ϕ=p/q
pbc=true
tx=ty=0
param_dict=Dict{}()
param_dict[t]=t
param_dict[ϕ]=ϕ
N1 = 10
N2 = 10
dx=2*pi/N1
dy=dx
Tx=collect(range(start=0, stop=2*pi-dx, step=dx))
Ty=collect(range(start=0, stop=2*pi-dy, step=dy));

In [2]:
using NBInclude
using OffsetArrays
@nbinclude("../../Single Particle/Lattice_Basis_Neighbors_Hops.ipynb"; regex=r"#.*executeme")
@nbinclude("../Chern_parameter_space_sum.ipynb")

calc_F12 (generic function with 1 method)

In [3]:
using LinearAlgebra
dimH=9

9

In [12]:
function calc_states_hofstadter(Tx, Ty, dimH)
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            tx = Tx[it1]
            ty = Ty[it2]
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(get_Hofstadter_hamiltonian(sites,neighbors_list,
                                        site_number,site_coord,
                                        1,LL,param_dict,tx, ty,pbc))
        end
    end
    return UUA
end

calc_states_hofstadter (generic function with 1 method)

In [13]:
using OffsetArrays
function calc_link_var(ψ1, ψ2)
    ψ1 = OffsetArrays.no_offset_view(ψ1)
    ψ2 = OffsetArrays.no_offset_view(ψ2)
    s = det(ψ1'*ψ2)
    return s/abs(s)
end

calc_link_var (generic function with 1 method)

In [14]:
function calc_link_vars(UUA)
    N1,N2,__ = size(UUA)
    UU=zeros(Complex{Float64}, N1, N2, 2)
    UU=OffsetArray(UU, 0:N1-1, 0:N2-1, 0:1)
    xdir = OffsetArray([1, 0], -1)
    ydir = OffsetArray([0, 1], -1)
    dirs = [ xdir, ydir ]
    dirs = OffsetArray(dirs, 0:1)
    for it1 in 0:N1-1
        for it2 in 0:N2-1
            for (idir, vdir) in pairs(dirs)
                UU[it1,it2,idir] = calc_link_var(UUA[it1,it2,:,:],
                                                 UUA[mod(it1+vdir[0],N1),mod(it2+vdir[1],N2),:,:])
            end
        end
    end
    return UU 
end

calc_link_vars (generic function with 1 method)

In [15]:
function calc_F12(UU)
    F12 = log( UU[:,:,1]*
               circshift(UU[:,:,2],(-1,0))*
               conj(circshift(UU[:,:,1],(0,-1)))*
                UU[:,:,2])
    return F12
end

calc_F12 (generic function with 1 method)

In [16]:
# EEA, UUA = calc_states_hofstadter(Tx, Ty, dimH)
# UUA[1,1,:,1]

In [18]:
UUA = calc_states_hofstadter(Tx, Ty, dimH)
UUA = OffsetArray(UUA, 0:N1-1, 0:N2-1, 0:dimH-1, 0:dimH-1)
UU = calc_link_vars(UUA[:,:,:,0:2])
UU = OffsetArrays.no_offset_view(UU)
F12 = calc_F12(UU)
sum(F12)/2im*pi

1.6535981962073898 - 55.94728622965145im